In [6]:
import numpy as np
from tqdm import tqdm
import vtk
def load_from_binary(filename):
    with open(filename, "rb") as file:
        # 读取外部 vector 的长度
        outer_size = np.fromfile(file, dtype=np.uint64, count=1)[0]
        data = []

        for _ in range(outer_size):
            # 读取每个内部 vector 的长度
            inner_size = int(np.fromfile(file, dtype=np.uint64, count=1)[0])  # 转换为整数

            # 读取内部 vector 的数据
            inner_data = np.fromfile(file, dtype=np.float64, count=inner_size * 2)  # 保证 count 是整数
            inner_vector = inner_data.reshape((inner_size, 2))
            data.append(inner_vector)

    return data



Total length: 68966067


In [24]:
! pip install tqdm 

In [31]:
from tqdm import tqdm
def save_to_vtk(data, filename):
    # 创建一个 vtkPoints 对象来存储所有的点
    points = vtk.vtkPoints()
    
    # 创建一个 vtkCellArray 对象来存储所有的线
    lines = vtk.vtkCellArray()

    for traj in tqdm(data, desc="Processing trajectories"):
        # 检查轨迹的最后一个点是否为 (-1, -1)，如果是则移除
        if np.array_equal(traj[-1], np.array([-1, -1])):
            traj = traj[:-1]
        
        # # 如果移除后轨迹为空，跳过该轨迹
        # if len(traj) == 0:
        #     continue
        
        # 记录线的起始点索引
        line = vtk.vtkPolyLine()
        line.GetPointIds().SetNumberOfIds(len(traj))
        
        for i, point in enumerate(traj):
            # 将点添加到 vtkPoints
            point_id = points.InsertNextPoint(point[0], point[1], 0)
            line.GetPointIds().SetId(i, point_id)
        
        # 将线添加到 vtkCellArray
        lines.InsertNextCell(line)
    
    # 创建一个 vtkPolyData 对象来存储点和线
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetLines(lines)
    
    # 将数据写入 VTK 文件
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(polydata)
    # 设置为二进制格式 否则很大 =>2d 1.8G=>1.3G
    writer.SetFileTypeToBinary()
    writer.Write()


In [32]:

data = load_from_binary("dec_traj.bin")

total_length = 0
for inner_vector in data:
    total_length += inner_vector.shape[0]
print("Total length:", total_length)
save_to_vtk(data, "trajectories_dec.vtk")

Processing trajectories: 100%|██████████| 41504/41504 [01:22<00:00, 503.73it/s]
